# Bias Analysis Across Demographic Groups

This notebook analyzes model performance across different demographic groups to identify and measure bias.

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score

%matplotlib inline

## 1. Performance by Gender

In [ ]:
def analyze_performance_by_gender(results_df):
    """Analyze model performance by gender"""
    
    gender_groups = results_df.groupby('true_gender')
    
    metrics = []
    for gender, group in gender_groups:
        mae = np.mean(np.abs(group['predicted_age'] - group['true_age']))
        accuracy = accuracy_score(group['true_gender'], group['predicted_gender']) * 100
        
        metrics.append({
            'gender': gender,
            'age_mae': mae,
            'gender_accuracy': accuracy,
            'count': len(group)
        })
    
    metrics_df = pd.DataFrame(metrics)
    print("Performance by Gender:")
    print(metrics_df)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    axes[0].bar(metrics_df['gender'], metrics_df['age_mae'])
    axes[0].set_title('Age MAE by Gender')
    axes[0].set_ylabel('MAE (years)')
    
    axes[1].bar(metrics_df['gender'], metrics_df['gender_accuracy'])
    axes[1].set_title('Gender Classification Accuracy')
    axes[1].set_ylabel('Accuracy (%)')
    axes[1].axhline(y=95, color='r', linestyle='--', label='Target (95%)')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()
    
    return metrics_df

## 2. Performance by Ethnicity

In [ ]:
def analyze_performance_by_ethnicity(results_df):
    """Analyze model performance by ethnicity"""
    
    ethnicity_groups = results_df.groupby('ethnicity')
    
    metrics = []
    for ethnicity, group in ethnicity_groups:
        mae = np.mean(np.abs(group['predicted_age'] - group['true_age']))
        
        metrics.append({
            'ethnicity': ethnicity,
            'age_mae': mae,
            'count': len(group)
        })
    
    metrics_df = pd.DataFrame(metrics)
    print("Performance by Ethnicity:")
    print(metrics_df)
    
    plt.figure(figsize=(12, 6))
    plt.bar(metrics_df['ethnicity'], metrics_df['age_mae'])
    plt.title('Age MAE by Ethnicity')
    plt.ylabel('MAE (years)')
    plt.xticks(rotation=45)
    plt.axhline(y=4.0, color='r', linestyle='--', label='Target MAE (4.0)')
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    return metrics_df

## 3. Fairness Metrics

In [ ]:
def calculate_fairness_metrics(results_df):
    """Calculate fairness metrics"""
    
    # Demographic Parity
    gender_positive_rates = results_df.groupby('true_gender')['predicted_gender'].apply(
        lambda x: (x == 'male').mean()
    )
    demographic_parity = gender_positive_rates.max() - gender_positive_rates.min()
    
    print(f"Demographic Parity Difference: {demographic_parity:.4f}")
    print(f"  (Lower is better, < 0.1 is considered fair)\n")
    
    # Performance Disparity
    gender_mae = results_df.groupby('true_gender').apply(
        lambda x: np.mean(np.abs(x['predicted_age'] - x['true_age']))
    )
    performance_disparity = gender_mae.max() - gender_mae.min()
    
    print(f"Performance Disparity (Age MAE): {performance_disparity:.2f} years")
    print(f"  (Lower is better)\n")
    
    return {
        'demographic_parity': demographic_parity,
        'performance_disparity': performance_disparity
    }

## 4. Intersectional Analysis

In [ ]:
def intersectional_analysis(results_df):
    """Analyze performance across intersectional groups"""
    
    intersectional = results_df.groupby(['true_gender', 'ethnicity']).apply(
        lambda x: pd.Series({
            'age_mae': np.mean(np.abs(x['predicted_age'] - x['true_age'])),
            'count': len(x)
        })
    ).reset_index()
    
    pivot = intersectional.pivot(index='ethnicity', columns='true_gender', values='age_mae')
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(pivot, annot=True, fmt='.2f', cmap='YlOrRd')
    plt.title('Age MAE by Gender and Ethnicity')
    plt.tight_layout()
    plt.show()
    
    return intersectional

## 5. Run Analysis

**Note:** Load your evaluation results with demographic labels before running.

In [ ]:
# Example usage
# results_df = pd.read_csv('evaluation_results.csv')
# gender_metrics = analyze_performance_by_gender(results_df)
# ethnicity_metrics = analyze_performance_by_ethnicity(results_df)
# fairness_metrics = calculate_fairness_metrics(results_df)
# intersectional_metrics = intersectional_analysis(results_df)